In [1]:
#Importierte Module/Funktionen
import requests
from bs4 import BeautifulSoup
import openpyxl
from openpyxl.styles import PatternFill
from openpyxl.styles import Border, Side


In [2]:
#Liste der URLs
url1 = "https://capital.com/de/apple-inc-aktienkurs"
url2 = "https://capital.com/de/nvidia-aktienkurs"
url3 = "https://capital.com/de/bank-of-america-aktienkurs"
url_test=[url1, url2, url3]

#Überprüfung ob Scraping auf den jeweiligen Seiten möglich ist
for r in url_test:
    print(r, ":")
    check=requests.get(r)
    
    if check.status_code==200:
        print("Scraping is possible")
    else:
        print("Scraping is limited")
        print()
        print("Please use a different URL!")
        break
        

https://capital.com/de/apple-inc-aktienkurs :
Scraping is possible
https://capital.com/de/nvidia-aktienkurs :
Scraping is possible
https://capital.com/de/bank-of-america-aktienkurs :
Scraping is possible


In [3]:
#Starten einer Excel.Datei
workbook = openpyxl.Workbook()
sheet = workbook.active

#Bearbeitung der Spaltenbreite
columns = ["A", "B"]
column_width = 20  

for column in columns:
    sheet.column_dimensions[column].width = column_width

# Schattierung der Zellen wird bearbeitet
cell_color = PatternFill(start_color="3A9EF8", end_color="3A9EF8", fill_type="solid")
cell1= sheet["A1"]
cell2= sheet["B1"]
cell1.fill=cell_color
cell2.fill=cell_color

# Rahmenlinien der Überschrift-Zellen wird eingestellt
border_style = Side(border_style="medium", color="000000")
border = Border(top=border_style, right=border_style, bottom=border_style, left=border_style)
cell1.border = border
cell2.border = border

# Rahmenlinien der unteren Zellen wird eingestellt
border_style = Side(border_style="thin", color="000000")
border_right = Border(right=border_style)
cell_range = sheet["A2:A4"]
for row in cell_range:
    for cell in row:
        cell.border = border_right


In [4]:
# Eintragung  der Spaltenüberschriften 
sheet.cell(row=1, column=1, value="Name:")
sheet.cell(row=1, column=2, value="Kurs:")

#Index damit die Werte ab der zweiten Zeile eingetragen werden
row_index = 2

#Angegebene URL wird verwendet, um den HTML-Code abzubilden
for p in url_test:
    page=requests.get(p)
    source_c=BeautifulSoup(page.text, 'html.parser')
    
    #Ausfilterung der Aktiennamen
    for x_tag in source_c.find_all("h1"):
        headline=x_tag.text.split("-")[0].strip()
        
        #Eintragung in Excel
        for i in range(len(headline)):
            zelle_h = sheet.cell(row=row_index, column=1)
            zelle_h.value = headline
            row_index += 1
            print(zelle_h.value)
            break
            

Apple Inc
NVIDIA
Bank of America


In [5]:
#Index damit die Werte ab der zweiten Zeile eingetragen werden
row_index = 2

#Angegebene URL wird verwendet, um den HTML-Code abzubilden
for p in url_test:
    page=requests.get(p)
    source_c=BeautifulSoup(page.text, 'html.parser')
    
    #Ausfilterung des Aktienkurs
    for data in source_c.find_all("b"):
        text = data.text.strip()
        
        # Überprüfung ob der Text eine Zahl bzw. Float ist
        if text.replace(".", "", 1).isdigit():  
            value = float(text)
            print(value)
            
            #Eintragung in Excel
            zelle_v = sheet.cell(row=row_index, column=2)
            zelle_v.value = value
            row_index += 1
            break
            
        else:
            print("ERROR keine ZAHL")
            break

ERROR keine ZAHL
ERROR keine ZAHL
ERROR keine ZAHL


In [6]:
#Speicherung der Excel 
workbook.save('Aktuelle Daten.xlsx')
